In [11]:
from pytorch_tabnet.tab_model import TabNetClassifier
from helpers.preproccesing_tabnet_attack import get_weights, normalize, get_bounds

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.datasets import fetch_openml

import pandas as pd
import numpy as np
np.random.seed(0)

# Pytorch
import torch
import torch.nn as nn
from torch.autograd import Variable

import tqdm
from tqdm import tqdm
from tqdm import tqdm


import os
from pathlib import Path

from matplotlib import pyplot as plt
%matplotlib inline

# Get Credit-g Dataset

In [12]:
dataset = 'credit-g'
dataset_name = 'credit-g'
target = 'target' 

dataset = fetch_openml(dataset)

train = pd.DataFrame(data= np.c_[dataset['data'], dataset[target]],
                  columns= dataset['feature_names'] + [target]) 

 # Renaming target for training later
train[target] = train[target].apply(lambda x : 0.0 if x == 'bad' or x == 0.0 else 1)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/datasets/_openml.py:421: UserWarning: Multiple active versions of the dataset matching the name credit-g exist. Versions may be fundamentally different, returning version 1.
  " {version}.".format(name=name, version=res[0]["version"])


# Split Data

In [13]:
train_set = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(train.shape[0],))

train_indices = train[train_set=="train"].index
valid_indices = train[train_set=="valid"].index
test_indices = train[train_set=="test"].index


# Preproccessing Label encode categorical features.

In [14]:
nunique = train.nunique()
types = train.dtypes

categorical_columns = []
categorical_dims =  {}
for col in train.columns:
    if types[col] == 'object' or nunique[col] < 200:
        print(col, train[col].nunique())
        l_enc = LabelEncoder()
        train[col] = train[col].fillna("VV_likely")
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        train.fillna(train.loc[train_indices, col].mean(), inplace=True)

checking_status 4
duration 33
credit_history 5
purpose 10
credit_amount 921
savings_status 5
employment 5
installment_commitment 4
personal_status 4
other_parties 3
residence_since 4
property_magnitude 4
age 53
other_payment_plans 3
housing 3
existing_credits 4
job 4
num_dependents 2
own_telephone 2
foreign_worker 2
target 2


# Define categorical features for categorical embeddings

In [15]:
# unused_feat = ['Set']

unused_feat = []

features = [ col for col in train.columns if col not in unused_feat+[target]] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

# Network parameters

In [16]:
tabnet_params = {"cat_idxs":cat_idxs,
                 "cat_dims":cat_dims,
                 "cat_emb_dim":1,
                 "optimizer_fn":torch.optim.Adam,
                 "optimizer_params":dict(lr=2e-3),
                 "scheduler_params":{"step_size":20, # how to use learning rate scheduler
                                 "gamma":0.9},
                 "scheduler_fn":torch.optim.lr_scheduler.StepLR,
                 "mask_type":'entmax' # "sparsemax"
                }

clf = TabNetClassifier(**tabnet_params
                      )



In [17]:
SEED = 0
# Compute the bounds for clipping
bounds = get_bounds(train)

# Normalize the data
#scaler, train, bounds = normalize(train, target, features, bounds)

# Compute the weights modelizing the expert's knowledge
weights = get_weights(train, target)

# Training

In [18]:
X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]

X_test = train[features].values[test_indices]
y_test = train[target].values[test_indices]

In [19]:
max_epochs = 1000 if not os.getenv("CI", False) else 2

In [20]:
from pytorch_tabnet.augmentations import ClassificationSMOTE
aug = ClassificationSMOTE(p=0.2)

In [21]:
## This illustrates the warm_start=False behaviour
save_history = []
for _ in range(2):
    clf.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_name=['train', 'valid'],
        eval_metric=['auc'],
        max_epochs=max_epochs , patience=20,
        batch_size=1024, virtual_batch_size=128,
        num_workers=0,
        weights=1,
        drop_last=False,
        augmentations=aug, #aug, None
    )
    save_history.append(clf.history["valid_auc"])

assert(np.all(np.array(save_history[0]==np.array(save_history[1]))))

epoch 0  | loss: 1.12169 | train_auc: 0.49059 | valid_auc: 0.41205 |  0:00:00s
epoch 1  | loss: 1.11225 | train_auc: 0.52627 | valid_auc: 0.43017 |  0:00:00s
epoch 2  | loss: 1.02725 | train_auc: 0.53272 | valid_auc: 0.45416 |  0:00:00s
epoch 3  | loss: 0.95978 | train_auc: 0.5317  | valid_auc: 0.46482 |  0:00:00s
epoch 4  | loss: 0.95134 | train_auc: 0.52825 | valid_auc: 0.46802 |  0:00:00s
epoch 5  | loss: 0.95533 | train_auc: 0.53019 | valid_auc: 0.47228 |  0:00:00s
epoch 6  | loss: 0.96258 | train_auc: 0.53373 | valid_auc: 0.48134 |  0:00:00s
epoch 7  | loss: 0.91441 | train_auc: 0.53673 | valid_auc: 0.49627 |  0:00:01s
epoch 8  | loss: 0.93052 | train_auc: 0.5393  | valid_auc: 0.50533 |  0:00:01s
epoch 9  | loss: 0.85057 | train_auc: 0.54247 | valid_auc: 0.51812 |  0:00:01s
epoch 10 | loss: 0.84792 | train_auc: 0.54479 | valid_auc: 0.52665 |  0:00:01s
epoch 11 | loss: 0.80767 | train_auc: 0.54851 | valid_auc: 0.52985 |  0:00:01s
epoch 12 | loss: 0.82005 | train_auc: 0.54813 | vali

epoch 104| loss: 0.66367 | train_auc: 0.63726 | valid_auc: 0.60181 |  0:00:11s
epoch 105| loss: 0.63714 | train_auc: 0.63889 | valid_auc: 0.60235 |  0:00:11s
epoch 106| loss: 0.65534 | train_auc: 0.64    | valid_auc: 0.60181 |  0:00:11s
epoch 107| loss: 0.65302 | train_auc: 0.6414  | valid_auc: 0.60128 |  0:00:11s
epoch 108| loss: 0.65729 | train_auc: 0.64237 | valid_auc: 0.59808 |  0:00:12s
epoch 109| loss: 0.65786 | train_auc: 0.64352 | valid_auc: 0.59915 |  0:00:12s
epoch 110| loss: 0.6498  | train_auc: 0.64193 | valid_auc: 0.59488 |  0:00:12s
epoch 111| loss: 0.66234 | train_auc: 0.6417  | valid_auc: 0.59808 |  0:00:12s
epoch 112| loss: 0.63517 | train_auc: 0.64205 | valid_auc: 0.59648 |  0:00:12s
epoch 113| loss: 0.65684 | train_auc: 0.64161 | valid_auc: 0.59701 |  0:00:12s
epoch 114| loss: 0.67401 | train_auc: 0.64109 | valid_auc: 0.60181 |  0:00:12s
epoch 115| loss: 0.63866 | train_auc: 0.64122 | valid_auc: 0.60501 |  0:00:12s
epoch 116| loss: 0.64437 | train_auc: 0.64264 | vali

epoch 208| loss: 0.61677 | train_auc: 0.71335 | valid_auc: 0.63646 |  0:00:24s
epoch 209| loss: 0.61667 | train_auc: 0.71498 | valid_auc: 0.63699 |  0:00:24s
epoch 210| loss: 0.61685 | train_auc: 0.71593 | valid_auc: 0.63273 |  0:00:24s
epoch 211| loss: 0.59173 | train_auc: 0.71481 | valid_auc: 0.63273 |  0:00:24s
epoch 212| loss: 0.59153 | train_auc: 0.71506 | valid_auc: 0.63273 |  0:00:24s
epoch 213| loss: 0.60921 | train_auc: 0.71639 | valid_auc: 0.63539 |  0:00:24s
epoch 214| loss: 0.61612 | train_auc: 0.71842 | valid_auc: 0.63806 |  0:00:24s
epoch 215| loss: 0.59058 | train_auc: 0.71956 | valid_auc: 0.63539 |  0:00:24s
epoch 216| loss: 0.60639 | train_auc: 0.7207  | valid_auc: 0.6338  |  0:00:24s
epoch 217| loss: 0.60112 | train_auc: 0.72135 | valid_auc: 0.63486 |  0:00:25s
epoch 218| loss: 0.59969 | train_auc: 0.72294 | valid_auc: 0.63806 |  0:00:25s
epoch 219| loss: 0.60936 | train_auc: 0.72133 | valid_auc: 0.63593 |  0:00:25s
epoch 220| loss: 0.60694 | train_auc: 0.72077 | vali

/Users/nazaraburas/Desktop/dl_project/TabSec/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


epoch 1  | loss: 1.11225 | train_auc: 0.52627 | valid_auc: 0.43017 |  0:00:00s
epoch 2  | loss: 1.02725 | train_auc: 0.53272 | valid_auc: 0.45416 |  0:00:00s
epoch 3  | loss: 0.95978 | train_auc: 0.5317  | valid_auc: 0.46482 |  0:00:00s
epoch 4  | loss: 0.95134 | train_auc: 0.52825 | valid_auc: 0.46802 |  0:00:00s
epoch 5  | loss: 0.95533 | train_auc: 0.53019 | valid_auc: 0.47228 |  0:00:00s
epoch 6  | loss: 0.96258 | train_auc: 0.53373 | valid_auc: 0.48134 |  0:00:00s
epoch 7  | loss: 0.91441 | train_auc: 0.53673 | valid_auc: 0.49627 |  0:00:01s
epoch 8  | loss: 0.93052 | train_auc: 0.5393  | valid_auc: 0.50533 |  0:00:01s
epoch 9  | loss: 0.85057 | train_auc: 0.54247 | valid_auc: 0.51812 |  0:00:01s
epoch 10 | loss: 0.84792 | train_auc: 0.54479 | valid_auc: 0.52665 |  0:00:01s
epoch 11 | loss: 0.80767 | train_auc: 0.54851 | valid_auc: 0.52985 |  0:00:01s
epoch 12 | loss: 0.82005 | train_auc: 0.54813 | valid_auc: 0.53785 |  0:00:01s
epoch 13 | loss: 0.84791 | train_auc: 0.55138 | vali

epoch 105| loss: 0.63714 | train_auc: 0.63889 | valid_auc: 0.60235 |  0:00:15s
epoch 106| loss: 0.65534 | train_auc: 0.64    | valid_auc: 0.60181 |  0:00:15s
epoch 107| loss: 0.65302 | train_auc: 0.6414  | valid_auc: 0.60128 |  0:00:15s
epoch 108| loss: 0.65729 | train_auc: 0.64237 | valid_auc: 0.59808 |  0:00:15s
epoch 109| loss: 0.65786 | train_auc: 0.64352 | valid_auc: 0.59915 |  0:00:15s
epoch 110| loss: 0.6498  | train_auc: 0.64193 | valid_auc: 0.59488 |  0:00:16s
epoch 111| loss: 0.66234 | train_auc: 0.6417  | valid_auc: 0.59808 |  0:00:16s
epoch 112| loss: 0.63517 | train_auc: 0.64205 | valid_auc: 0.59648 |  0:00:16s
epoch 113| loss: 0.65684 | train_auc: 0.64161 | valid_auc: 0.59701 |  0:00:16s
epoch 114| loss: 0.67401 | train_auc: 0.64109 | valid_auc: 0.60181 |  0:00:16s
epoch 115| loss: 0.63866 | train_auc: 0.64122 | valid_auc: 0.60501 |  0:00:16s
epoch 116| loss: 0.64437 | train_auc: 0.64264 | valid_auc: 0.60608 |  0:00:16s
epoch 117| loss: 0.63941 | train_auc: 0.64261 | vali

epoch 209| loss: 0.61667 | train_auc: 0.71498 | valid_auc: 0.63699 |  0:00:27s
epoch 210| loss: 0.61685 | train_auc: 0.71593 | valid_auc: 0.63273 |  0:00:27s
epoch 211| loss: 0.59173 | train_auc: 0.71481 | valid_auc: 0.63273 |  0:00:27s
epoch 212| loss: 0.59153 | train_auc: 0.71506 | valid_auc: 0.63273 |  0:00:27s
epoch 213| loss: 0.60921 | train_auc: 0.71639 | valid_auc: 0.63539 |  0:00:27s
epoch 214| loss: 0.61612 | train_auc: 0.71842 | valid_auc: 0.63806 |  0:00:27s
epoch 215| loss: 0.59058 | train_auc: 0.71956 | valid_auc: 0.63539 |  0:00:28s
epoch 216| loss: 0.60639 | train_auc: 0.7207  | valid_auc: 0.6338  |  0:00:28s
epoch 217| loss: 0.60112 | train_auc: 0.72135 | valid_auc: 0.63486 |  0:00:28s
epoch 218| loss: 0.59969 | train_auc: 0.72294 | valid_auc: 0.63806 |  0:00:28s
epoch 219| loss: 0.60936 | train_auc: 0.72133 | valid_auc: 0.63593 |  0:00:28s
epoch 220| loss: 0.60694 | train_auc: 0.72077 | valid_auc: 0.63699 |  0:00:28s

Early stopping occurred at epoch 220 with best_epoc

/Users/nazaraburas/Desktop/dl_project/TabSec/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [22]:
preds = clf.predict_proba(X_test)
test_auc = roc_auc_score(y_score=preds[:,1], y_true=y_test)


preds_valid = clf.predict_proba(X_valid)
valid_auc = roc_auc_score(y_score=preds_valid[:,1], y_true=y_valid)

print(f"BEST VALID SCORE FOR {dataset_name} : {clf.best_cost}")
print(f"FINAL TEST SCORE FOR {dataset_name} : {test_auc}")

BEST VALID SCORE FOR credit-g : 0.6481876332622601
FINAL TEST SCORE FOR credit-g : 0.6241426611796982


In [23]:
# save tabnet model
saving_path_name = "./tabnet_model_test_credit_g"
saved_filepath = clf.save_model(saving_path_name)

Successfully saved model at ./tabnet_model_test_credit_g.zip


In [24]:
# define new model with basic parameters and load state dict weights
loaded_clf = TabNetClassifier()
loaded_clf.load_model("./tabnet_model_test_credit_g.zip")

/Users/nazaraburas/Desktop/dl_project/TabSec/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


# Predictions

In [25]:
dataset_name = 'credit-g'

preds = clf.predict_proba(X_test)
test_auc = roc_auc_score(y_score=preds[:,1], y_true=y_test)


preds_valid = clf.predict_proba(X_valid)
valid_auc = roc_auc_score(y_score=preds_valid[:,1], y_true=y_valid)

print("TabNet")
print(f"BEST VALID SCORE FOR {dataset_name} : {clf.best_cost}")
print(f"FINAL TEST SCORE FOR {dataset_name} : {test_auc}")

TabNet
BEST VALID SCORE FOR credit-g : 0.6481876332622601
FINAL TEST SCORE FOR credit-g : 0.6241426611796982


In [26]:
# check that best weights are used
assert np.isclose(valid_auc, np.max(clf.history['valid_auc']), atol=1e-6)

In [27]:
# sanity check
clf.predict(X_test)

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0])

In [28]:
# save tabnet model
saving_path_name = "./tabnet_model_credit_test_1"
saved_filepath = clf.save_model(saving_path_name)

Successfully saved model at ./tabnet_model_credit_test_1.zip


In [29]:
loaded_preds = loaded_clf.predict_proba(X_test)
loaded_test_auc = roc_auc_score(y_score=loaded_preds[:,1], y_true=y_test)

print(f"FINAL TEST SCORE FOR {dataset_name} : {loaded_test_auc}")

FINAL TEST SCORE FOR credit-g : 0.6241426611796982


In [30]:
# sanity check
loaded_clf.predict(X_test)

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0])

# Generate Adversarial Examples

In [31]:
SEED = 11

settings = {'MaxIters': 500,
            'Alpha': 0.1,
            'Lambda': 8.5}

#settings['TestData'] = train[features].iloc[list(test_indices)]
settings['TestData'] = train.iloc[list(test_indices)]
settings['TestData'] = settings['TestData'].sample(n=10, random_state=SEED)
settings['TestDataOrig'] = settings['TestData']
settings['TestData'] = settings['TestData'][features]

# Sub sample

settings['Target'] = target
settings['FeatureNames'] = features
settings['Weights'] = weights
settings['Bounds'] = bounds


loaded_clf.predict(settings['TestData'].values)


array([1, 1, 1, 0, 1, 0, 0, 0, 1, 0])

In [32]:
def gen_adv(model, config, method):
    extra_cols = ['orig_pred', 'adv_pred', 'iters']
    df_test = config['TestData']
    feature_names = config['FeatureNames']
    weights = config['Weights']
    bounds = config['Bounds']
    maxiters = config['MaxIters']
    alpha = config['Alpha']
    lambda_ = config['Lambda']
    target = config['Target']
    
    results = np.zeros((len(df_test), len(feature_names) + 1))
            
    i = -1
    n_samples = 0
    n_success = 0
    pert_norms = []
    weighted_pert_norms = []
    
    
    for _, row in tqdm(df_test.iterrows(), total=df_test.shape[0], desc="{}".format(method)):
        i += 1
        
        x_tensor = row.values
        
        n_samples += 1
        
        if method == 'LowProFool':
            orig_pred, adv_pred, x_adv, loop_i = lowProFool(x_tensor, model, weights, bounds,
                                                             maxiters, alpha, lambda_)
        elif method == 'Deepfool':
            orig_pred, adv_pred, x_adv, loop_i = deepfool(x_tensor, model, maxiters, alpha,
                                                          bounds, weights=[])
        else:
            raise Exception("Invalid method", method)

        #pert = x_adv - x_tensor.numpy()
        pert = x_adv - x_tensor

        if orig_pred != adv_pred:
            n_success += 1
            pert_norms.append(np.linalg.norm(pert))
            weighted_pert_norms.append(np.linalg.norm(weights * pert))

        results[i] = np.append(x_adv, orig_pred)
    df = pd.DataFrame(results, index=df_test.index, columns=feature_names + [target])
    return df, n_success/n_samples, np.mean(weighted_pert_norms), np.std(weighted_pert_norms),\
                                    np.mean(pert_norms), np.std(pert_norms)


# Clipping function
def clip(current, low_bound, up_bound):
    assert(len(current) == len(up_bound) and len(low_bound) == len(up_bound))
    low_bound = torch.FloatTensor(low_bound)
    up_bound = torch.FloatTensor(up_bound)
    clipped = torch.max(torch.min(current, up_bound), low_bound)
    return clipped

# LowProFool

In [47]:
def lowProFool(x, model, weights, bounds, maxiters, alpha, lambda_):
    """
    Generates an adversarial examples x' from an original sample x

    :param x: tabular sample
    :param model: neural network
    :param weights: feature importance vector associated with the dataset at hand
    :param bounds: bounds of the datasets with respect to each feature
    :param maxiters: maximum number of iterations ran to generate the adversarial examples
    :param alpha: scaling factor used to control the growth of the perturbation
    :param lambda_: trade off factor between fooling the classifier and generating imperceptible adversarial example
    :return: original label prediction, final label prediction, adversarial examples x', iteration at which the class changed
    """

    r = Variable(torch.FloatTensor(1e-4 * np.ones(x.shape)), requires_grad=True) 
    v = torch.FloatTensor(np.array(weights))
    
    x_into_model = np.expand_dims(x, axis=0)
    x = torch.tensor(x)

    output = model.predict_proba(x_into_model) #outputs -> numpy array [[x, 1-x]]
    output = Variable(torch.tensor(output[0]), requires_grad=True)
    
    orig_pred = output.max(0, keepdim=True)[1].cpu().numpy()
    target_pred = np.abs(1 - orig_pred)
    
    target = np.array([0., 1.]) if target_pred == 1 else np.array([1., 0.])
    target = Variable(torch.tensor(target, requires_grad=False)) 
    
    lambda_ = torch.tensor([lambda_])
    
    bce = nn.BCELoss()
    l1 = lambda v, r: torch.sum(torch.abs(v * r)) #L1 norm
    l2 = lambda v, r: torch.sqrt(torch.sum(torch.mul(v * r,v * r))) #L2 norm

    best_norm_weighted = np.inf
    best_pert_x = x
    
    loop_i, loop_change_class = 0, 0
    while loop_i < maxiters: 
        if r.grad is not None:
            r.grad.zero_()
            
        # make sure same type tensors
        output = output.type(torch.FloatTensor)
        target = target.type(torch.FloatTensor)
        
        # Computing loss
        loss_1 = bce(output, target)
        loss_2 = l2(v, r)
        loss = (loss_1 + lambda_ * loss_2)

        # Get the gradient
        loss.backward(retain_graph=True)
        grad_r = r.grad.data.cpu().numpy().copy()

        # Guide perturbation to the negative of the gradient
        ri = - grad_r
    
        # limit huge step
        ri *= alpha

        # Adds new perturbation to total perturbation
        r = r.clone().detach().cpu().numpy() + ri
        
        # For later computation
        r_norm_weighted = np.sum(np.abs(r * weights))
        
        # Ready to feed the model
        r = Variable(torch.FloatTensor(r), requires_grad=True) 
        
        # Compute adversarial example
        xprime = x + r
    
        # Clip to stay in legitimate bounds
        xprime = clip(xprime, bounds[0], bounds[1])
        
        # model expects input of form [x1]
        xprime_into_model = torch.unsqueeze(xprime, 0)
        
        # Classify adversarial example
        output = model.predict_proba(xprime_into_model) #outputs -> numpy array [[x, 1-x]]
        
        output = Variable(torch.tensor(output[0]), requires_grad=True)
        
        output_pred = output.max(0, keepdim=True)[1].cpu().numpy()
        
        # Keep the best adverse at each iterations
        if output_pred != orig_pred and r_norm_weighted < best_norm_weighted:
            best_r = r
            best_norm_weighted = r_norm_weighted
            best_pert_x = xprime

        if output_pred == orig_pred:
            loop_change_class += 1
            
        loop_i += 1 
        
    # Clip at the end no matter what
    best_pert_x = clip(best_pert_x, bounds[0], bounds[1])
    best_pert_x_into_model = torch.unsqueeze(best_pert_x, 0)
    output = model.predict_proba(best_pert_x_into_model)
    output = Variable(torch.tensor(output[0]), requires_grad=False)
    output_pred = output.max(0, keepdim=True)[1].cpu().numpy()
    
    return orig_pred, output_pred, best_pert_x.clone().detach().cpu().numpy(), loop_change_class

In [48]:
# Generate adversarial examples
df_adv_lpf, *lpf_data = gen_adv(loaded_clf, settings, 'LowProFool')

LowProFool:   0%|          | 0/10 [00:00<?, ?it/s]

tensor([3.4475e+00, 1.7558e+00, 3.5510e-01, 7.1952e-01, 3.1112e-01, 4.0098e-01,
        3.0079e-03, 1.9763e-01, 3.0221e-01, 3.0493e-02, 3.3190e-04, 1.1624e-01,
        3.1290e-01, 1.0937e-01, 1.4064e-02, 7.8845e-02, 4.0448e-02, 3.4266e-04,
        5.0131e-02, 2.5398e-01])
tensor([-5.0555e+00, -1.3109e+00, -5.3500e-02, -2.1997e-01, -4.1051e-02,
        -6.8239e-02, -2.5697e-06, -1.6534e-02, -3.8731e-02, -3.8265e-04,
         9.4347e-08, -5.6998e-03, -4.1524e-02, -5.0427e-03, -7.8177e-05,
        -2.6115e-03, -6.7889e-04,  9.5836e-08, -1.0479e-03, -2.7337e-02])
tensor([ 5.2155e+00, -7.3278e-01, -1.0036e-01, -3.3728e-01, -7.8709e-02,
        -1.2507e-01, -5.6714e-06, -3.3465e-02, -7.4584e-02, -8.3462e-04,
         2.0846e-07, -1.1973e-02, -7.9544e-02, -1.0625e-02, -1.7172e-04,
        -5.5773e-03, -1.4744e-03,  2.1175e-07, -2.2664e-03, -5.3883e-02])
tensor([-5.3986e+00,  2.2025e-01, -3.0873e-02, -5.0383e-02, -2.5715e-02,
        -3.5981e-02, -2.6116e-06, -1.2582e-02, -2.4656e-02, -3.7437e

tensor([-5.4133e+00,  3.3170e-14, -1.7261e-05, -3.1013e-10, -4.1617e-05,
        -6.2972e-06, -2.4864e-06, -2.5940e-04, -4.9208e-05, -2.1469e-04,
         9.6206e-08, -5.4198e-04, -4.0228e-05, -5.5530e-04, -6.0832e-05,
        -5.4642e-04, -3.1181e-04,  9.7704e-08, -3.9561e-04, -1.1411e-04])
tensor([ 5.4133e+00,  1.5306e-14, -2.8851e-05, -4.2073e-10, -7.1141e-05,
        -1.0276e-05, -4.9121e-06, -4.6886e-04, -8.4497e-05, -4.1904e-04,
         1.9029e-07, -1.0177e-03, -6.8705e-05, -1.0460e-03, -1.1960e-04,
        -1.0437e-03, -6.0593e-04,  1.9325e-07, -7.6546e-04, -2.0069e-04])
tensor([-5.4133e+00, -3.9984e-15, -1.0110e-05, -8.0486e-11, -2.6297e-05,
        -3.3952e-06, -2.4767e-06, -1.9655e-04, -3.1563e-05, -2.0625e-04,
         9.6165e-08, -4.6281e-04, -2.5343e-05, -4.7882e-04, -5.9724e-05,
        -4.9167e-04, -2.9560e-04,  9.7661e-08, -3.7019e-04, -7.9194e-05])
tensor([ 5.4133e+00, -1.8449e-15, -1.6899e-05, -1.0919e-10, -4.4953e-05,
        -5.5402e-06, -4.8929e-06, -3.5525e-04, -

tensor([-5.4133e+00,  6.5746e-29, -3.3145e-09, -1.3144e-19, -2.6889e-08,
        -3.2117e-10, -2.3354e-06, -3.0621e-06, -4.0390e-08, -1.1303e-04,
         9.5546e-08, -4.3317e-05, -2.4771e-08, -5.1866e-05, -4.5331e-05,
        -1.0090e-04, -1.3272e-04,  9.7012e-08, -1.3672e-04, -3.3063e-07])
tensor([ 5.4133e+00,  3.0333e-29, -5.5396e-09, -1.7830e-19, -4.5962e-08,
        -5.2405e-10, -4.6136e-06, -5.5343e-06, -6.9350e-08, -2.2059e-04,
         1.8897e-07, -8.1333e-05, -4.2303e-08, -9.7693e-05, -8.9117e-05,
        -1.9272e-04, -2.5789e-04,  1.9187e-07, -2.6452e-04, -5.8146e-07])
tensor([-5.4133e+00, -7.9233e-30, -1.9415e-09, -3.4114e-20, -1.6991e-08,
        -1.7316e-10, -2.3263e-06, -2.3202e-06, -2.5907e-08, -1.0858e-04,
         9.5505e-08, -3.6989e-05, -1.5606e-08, -4.4722e-05, -4.4505e-05,
        -9.0792e-05, -1.2582e-04,  9.6969e-08, -1.2794e-04, -2.2946e-07])
tensor([ 5.4133e+00, -3.6555e-30, -3.2448e-09, -4.6276e-20, -2.9043e-08,
        -2.8255e-10, -4.5955e-06, -4.1934e-06, -

tensor([ 5.4133e+00,  0.0000e+00, -6.2314e-13, -1.9621e-29, -1.8766e-11,
        -1.4412e-14, -4.3164e-06, -4.9503e-08, -3.6513e-11, -1.1157e-04,
         1.8759e-07, -5.5508e-06, -1.6412e-11, -7.8679e-06, -6.5197e-05,
        -3.2021e-05, -1.0406e-04,  1.9042e-07, -8.5544e-05, -1.1693e-09])
tensor([-5.4133e+00,  0.0000e+00, -2.1840e-13, -3.7541e-30, -6.9376e-12,
        -4.7623e-15, -2.1765e-06, -2.0754e-08, -1.3640e-11, -5.4917e-05,
         9.4807e-08, -2.5244e-06, -6.0544e-12, -3.6018e-06, -3.2560e-05,
        -1.5086e-05, -5.0768e-05,  9.6237e-08, -4.1374e-05, -4.6146e-10])
tensor([ 5.4133e+00,  0.0000e+00, -3.6500e-13, -5.0925e-30, -1.1858e-11,
        -7.7706e-15, -4.2995e-06, -3.7509e-08, -2.3420e-11, -1.0718e-04,
         1.8750e-07, -4.7399e-06, -1.0339e-11, -6.7843e-06, -6.4009e-05,
        -2.8813e-05, -9.8649e-05,  1.9033e-07, -8.0048e-05, -8.1154e-10])
tensor([-5.4133e+00,  0.0000e+00, -1.2793e-13, -9.7435e-31, -4.3838e-12,
        -2.5677e-15, -2.1680e-06, -1.5725e-08, -

...

tensor([ 5.4133e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -5.8740e-41,
        -0.0000e+00, -2.4178e-06, -7.2652e-26, -1.0457e-39, -2.9519e-07,
         1.7598e-07, -3.9269e-16, -3.2928e-41, -2.3529e-15, -4.2911e-06,
        -5.2406e-12, -3.8532e-08,  1.7826e-07, -4.6124e-09, -3.9065e-33])
tensor([-5.4133e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -2.1715e-41,
        -0.0000e+00, -1.2191e-06, -3.0459e-26, -3.9063e-40, -1.4530e-07,
         8.8939e-08, -1.7859e-16, -1.2146e-41, -1.0771e-15, -2.1430e-06,
        -2.4689e-12, -1.8799e-08,  9.0094e-08, -2.2308e-09, -1.5416e-33])
tensor([ 5.4133e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -3.7126e-41,
        -0.0000e+00, -2.4083e-06, -5.5049e-26, -6.7071e-40, -2.8359e-07,
         1.7590e-07, -3.3532e-16, -2.0748e-41, -2.0288e-15, -4.2129e-06,
        -4.7155e-12, -3.6529e-08,  1.7818e-07, -4.3161e-09, -2.7112e-33])
tensor([-5.4133e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -1.3724e-41,
        -0.0000e+00, -1.2144e-06, -2.3079e-26, -

tensor([-5.4133e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -1.1210e-44,
        -0.0000e+00, -1.1406e-06, -2.7245e-28, -2.0739e-43, -7.3489e-08,
         8.8289e-08, -1.2188e-17, -0.0000e+00, -8.6748e-17, -1.5678e-06,
        -4.1023e-13, -7.5855e-09,  8.9414e-08, -7.2143e-10, -3.1002e-36])
tensor([ 5.4133e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -1.9618e-44,
        -0.0000e+00, -2.2532e-06, -4.9240e-28, -3.5032e-43, -1.4343e-07,
         1.7461e-07, -2.2885e-17, -0.0000e+00, -1.6340e-16, -3.0821e-06,
        -7.8351e-13, -1.4739e-08,  1.7684e-07, -1.3958e-09, -5.4523e-36])
tensor([-5.4133e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -1.1210e-44,
        -0.0000e+00, -1.1361e-06, -2.0644e-28, -1.2612e-43, -7.0600e-08,
         8.8251e-08, -1.0408e-17, -0.0000e+00, -7.4801e-17, -1.5392e-06,
        -3.6913e-13, -7.1912e-09,  8.9374e-08, -6.7508e-10, -2.1517e-36])
tensor([ 5.4133e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -1.9618e-44,
        -0.0000e+00, -2.2444e-06, -3.7310e-28, -

tensor([ 5.4133e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
        -0.0000e+00, -2.1081e-06, -4.4044e-30, -0.0000e+00, -7.2540e-08,
         1.7334e-07, -1.5618e-18, -0.0000e+00, -1.3160e-17, -2.2549e-06,
        -1.3019e-13, -5.9474e-09,  1.7550e-07, -4.5138e-10, -1.0965e-38])
tensor([-5.4133e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
        -0.0000e+00, -1.0630e-06, -1.8465e-30, -0.0000e+00, -3.5707e-08,
         8.7605e-08, -7.1030e-19, -0.0000e+00, -6.0243e-18, -1.1261e-06,
        -6.1333e-14, -2.9017e-09,  8.8700e-08, -2.1831e-10, -4.3270e-39])
tensor([ 5.4133e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
        -0.0000e+00, -2.0998e-06, -3.3373e-30, -0.0000e+00, -6.9689e-08,
         1.7326e-07, -1.3337e-18, -0.0000e+00, -1.1347e-17, -2.2138e-06,
        -1.1714e-13, -5.6382e-09,  1.7543e-07, -4.2238e-10, -7.6097e-39])
tensor([-5.4133e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
        -0.0000e+00, -1.0588e-06, -1.3991e-30, -

LowProFool: 100%|██████████| 10/10 [00:31<00:00,  3.11s/it]

tensor([-5.4133e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
        -0.0000e+00, -9.9451e-07, -1.6517e-32, -0.0000e+00, -1.8059e-08,
         8.6965e-08, -4.8476e-20, -0.0000e+00, -4.8518e-19, -8.2384e-07,
        -1.0191e-14, -1.1708e-09,  8.8030e-08, -7.0601e-11, -8.7049e-42])


# Attack Results

In [35]:
iterations = settings['MaxIters']
alpha = settings['Alpha']
lambdaa = settings['Lambda']
print(f'Iterations: {iterations} , Alpha: {alpha} , Lambda: {lambdaa}')
print('')
print('Before:')
print(loaded_clf.predict(settings['TestData'].values))
print('After:')
print(loaded_clf.predict(df_adv_lpf[features].values))
print('')


x = loaded_clf.predict(df_adv_lpf[features].values)

y = loaded_clf.predict(settings['TestData'].values)
result = sum(x != y) / len(x)
print(f"Adverserial generation success: {result}")

Iterations: 500 , Alpha: 0.1 , Lambda: 8.5

Before:
[1 1 1 0 1 0 0 0 1 0]
After:
[0 0 0 1 0 1 1 1 0 1]

Adverserial generation success: 1.0


# DeepFool

In [66]:
# Forked from https://github.com/LTS4/DeepFool
def deepfool(x_old, net, maxiters, alpha, bounds, weights=[], overshoot=0.002):
    """
    :param image: tabular sample
    :param net: network 
    :param maxiters: maximum number of iterations ran to generate the adversarial examples
    :param alpha: scaling factor used to control the growth of the perturbation
    :param bounds: bounds of the datasets with respect to each feature
    :param weights: feature importance vector associated with the dataset at hand
    :param overshoot: used as a termination criterion to prevent vanishing updates (default = 0.02).
    :return: minimal perturbation that fools the classifier, number of iterations that it required, new estimated_label and perturbed image
    """
    
    input_shape = x_old.shape
    x = np.expand_dims(x_old, axis=0)
    x = torch.FloatTensor(x)
    x = Variable(x, requires_grad=True)

    output = net.predict_proba(x)
    output = torch.tensor(output[0])
    output = torch.tensor(output, requires_grad=True)
    
    orig_pred = output.max(0, keepdim=True)[1] # get the index of the max log-probability
    
    origin = Variable(torch.tensor([orig_pred], requires_grad=False))

    
    I = []
    if orig_pred == 0:
        I = [0, 1]
    else:
        I = [1, 0]
        
    w = np.zeros(input_shape)
    r_tot = np.zeros(input_shape)
    
    k_i = origin
 
    loop_i = 0
    while torch.eq(k_i, origin) and loop_i < maxiters:
                
        # Origin class
        
        ### PROBLEM ###
        output[I[0]].backward(retain_graph=True)
        
        ### This is NONE ###
        grad_orig = x.grad.data.cpu().numpy().copy()
        
        # Target class
        if x.grad is not None:
            x.grad.zero_()
        output[I[1]].backward(retain_graph=True)
       
        cur_grad = x.grad.data.cpu().numpy().copy()
            
        # set new w and new f
        w = cur_grad - grad_orig
        f = (output[I[1]] - output[I[0]]).data.numpy()

        pert = abs(f)/np.linalg.norm(w.flatten())
    
        # compute r_i and r_tot
        # Added 1e-4 for numerical stability
        r_i =  (pert+1e-4) * w / np.linalg.norm(w)   
        
        if len(weights) > 0:
            r_i /= np.array(weights)

        # limit huge step
        r_i = alpha * r_i / np.linalg.norm(r_i) 
            
        r_tot = np.float32(r_tot + r_i)
        
        
        pert_x = x_old + (1 + overshoot) * torch.from_numpy(r_tot)

        if len(bounds) > 0:
            pert_x = clip(pert_x, bounds[0], bounds[1])
                
        x = Variable(pert_x, requires_grad=True)
 
        output = net.forward(x)
        
        k_i = torch.tensor(np.argmax(output.data.cpu().numpy().flatten()))
                    
        loop_i += 1

    r_tot = (1+overshoot)*r_tot    
    pert_x = clip(pert_x, bounds[0], bounds[1])

    return orig_pred, k_i, pert_x.clone().detach().cpu().numpy(), loop_i

In [67]:
# Generate adversarial examples
df_adv_lpf, *lpf_data = gen_adv(loaded_clf, settings, 'Deepfool')

Deepfool:   0%|          | 0/10 [00:00<?, ?it/s]

None


AttributeError: 'NoneType' object has no attribute 'data'

In [44]:
iterations = settings['MaxIters']
alpha = settings['Alpha']
lambdaa = settings['Lambda']
print(f'Iterations: {iterations} , Alpha: {alpha} , Lambda: {lambdaa}')
print('')
print('Before:')
print(loaded_clf.predict(settings['TestData'].values))
print('After:')
print(loaded_clf.predict(df_adv_lpf[features].values))
print('')


x = loaded_clf.predict(df_adv_lpf[features].values)

y = loaded_clf.predict(settings['TestData'].values)
result = sum(x != y) / len(x)
print(f"Adverserial generation success: {result}")

Iterations: 500 , Alpha: 0.1 , Lambda: 8.5

Before:
[1 1 1 0 1 0 0 0 1 0]
After:
[0 0 0 1 0 1 1 1 0 1]

Adverserial generation success: 1.0
